In [1]:
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import Input, Reshape, TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard
from music21 import *
import glob
import numpy as np
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.callbacks import History, ModelCheckpoint
from keras.optimizers import RMSprop
from datetime import datetime

/home/eurismar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/eurismar/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
path = '/home/eurismar/notebooks/Music_Generator_Demo/Pop_Music_Midi/'

In [3]:
files = glob.glob('{}/*.mid*'.format(path))

In [4]:
KEY_SIG_OFFSET = 0 
melody_corpus = []
for i, file in enumerate(files[:2]):
#for file in files:
    midi_stream = converter.parse(file)
    
    part0 = midi_stream[0]
    key_sig = part0.keySignature
    midi_stream.transpose(KEY_SIG_OFFSET - key_sig.tonic.pitchClass, inPlace=True)
    for m in part0.measures(1, None):
        if hasattr(m, 'flat'):
            for nr in m.flat.notesAndRests:
                pitch = nr.pitch.midi  if isinstance(nr, note.Note) else 0
                duration = float(nr.quarterLength)
                note_repr = (pitch, duration)
                melody_corpus.append(note_repr)

In [5]:
melody_set = set(melody_corpus)

In [6]:
notes_indices = {note: i for i, note in enumerate(melody_set)}
indices_notes = {i: note for i, note in enumerate(melody_set)}

In [7]:
corpus_size = len(melody_set)
print(corpus_size)

25


In [8]:
phrase_len = 20
step_size = 3

In [9]:
phrases = []
next_notes = []

In [10]:
for i in range(0, len(melody_corpus) - phrase_len, step_size):
    phrases.append(melody_corpus[i: i + phrase_len])
    next_notes.append(melody_corpus[i + phrase_len])
print('nb sequences:', len(phrases))

nb sequences: 43


In [11]:
# transform data into binary matrices
X = np.zeros((len(phrases), phrase_len, corpus_size), dtype=np.bool)
y = np.zeros((len(phrases), corpus_size), dtype=np.bool)
print(X.shape)
print(y.shape)

(43, 20, 25)
(43, 25)


In [12]:
for i, phrase in enumerate(phrases):
    for j, note in enumerate(phrase):
        X[i, j, notes_indices[note]] = 1
    y[i, notes_indices[next_notes[i]]] = 1

In [13]:
layer_size=128
batch_size = 128
nb_epoch = 400

In [14]:
corpus_size

25

In [15]:
phrase_len

20

In [16]:
def get_gen(G_in, phrase_len=phrase_len, corpus_size=corpus_size):
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(G_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size, return_sequences=True)(layer)
    layer = Dropout(0.2)(layer)
    layer = TimeDistributed(Dense(corpus_size))(layer)
    G_out = Activation('softmax')(layer)
    LSTM_out = Activation('softmax')(layer)
    G = Model(G_in, G_out)
    G.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return G, G_out
G_in = Input(shape=(phrase_len, corpus_size))
G, G_out = get_gen(G_in)
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           78848     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 25)            3225      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 25)            0         
Total para

In [17]:
def get_dis(D_in, phrase_len=phrase_len, corpus_size=corpus_size):
    
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(D_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size, return_sequences=True)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(2)(layer)
    D_out = Activation('sigmoid')(layer)
    D = Model(D_in, D_out)
    D.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return D, D_out    

In [18]:
D_in = Input(shape=(phrase_len, corpus_size))
#G_in = Input(shape=(phrase_len, corpus_size))

In [19]:
D, D_out = get_dis(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 128)           78848     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 20, 2)             258       
_________________________________________________________________
activation_3 (Activation)    (None, 20, 2)             0         
Total para

In [20]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

In [21]:
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in) # ENTRADA DA GAN
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

In [22]:
GAN_in = Input(shape=(phrase_len, corpus_size))
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
model_1 (Model)              (None, 20, 25)            213657    
_________________________________________________________________
model_2 (Model)              (None, 20, 2)             210690    
Total params: 424,347
Trainable params: 213,657
Non-trainable params: 210,690
_________________________________________________________________


In [23]:
X.shape

(43, 20, 25)

In [24]:
y.shape

(43, 25)

In [58]:
def sample_data():
    return X

In [59]:
sample_data()

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [43]:
def sample_data_and_gen(G,n_samples=10000):
    XT = X
    XN_noise = np.random.uniform(0,1,size=[1,20,25])
    XN = G.predict(XN_noise)
    XN.shape
    XX = np.concatenate((XT, XN))
    yy = np.zeros((2*n_samples, 2))
    yy[:n_samples, 1] = 1
    yy[n_samples:, 0] = 1
    return  XX, yy

In [56]:
for i, item in enumerate(X):
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

In [48]:
def pretrain(G, D, n_samples=10000, batch_size=32):
    X, y = sample_data_and_gen(G, n_samples=n_samples)
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

In [49]:
pretrain(G, D)

(44, 20, 25) (20000, 2)


ValueError: Error when checking target: expected activation_3 to have 3 dimensions, but got array with shape (20000, 2)

## arrumar daqui pra frente

In [59]:
data = np.random.randn(1,phrase_len,corpus_size)

In [60]:
phrase_len, corpus_size

(20, 25)

In [61]:
data.shape

(1, 20, 25)

In [62]:
X.shape

(43, 20, 25)

In [63]:
X = np.zeros((len(phrases), phrase_len, corpus_size), dtype=np.bool)

In [82]:
pred = GAN.predict(data)

In [83]:
pred.shape

(1, 20, 2)

In [ ]:
def sample_data(n_samples=10000, x_vals=np.arange(0, 5, .1), max_offset=100, mul_range=[1, 2]):
    vectors = []
    for i in range(n_samples):
        offset = np.random.random() * max_offset
        mul = mul_range[0] + np.random.random() * (mul_range[1] - mul_range[0])
        vectors.append(
            np.sin(offset + x_vals * mul) / 2 + .5
        )
    return np.array(vectors)

ax = pd.DataFrame(np.transpose(sample_data(5))).plot()

In [ ]:
def sample_data_and_gen(G, noise_dim=10, n_samples=10000):
    XT = sample_data(n_samples=n_samples)
    XN_noise = np.random.uniform(0, 1, size=[n_samples, noise_dim])
    XN = G.predict(XN_noise)
    X = np.concatenate((XT, XN))
    y = np.zeros((2*n_samples, 2))
    y[:n_samples, 1] = 1
    y[n_samples:, 0] = 1
    return X, y

In [ ]:
def pretrain(G, D, noise_dim=10, n_samples=10000, batch_size=32):
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

In [ ]:
pretrain(G, D)

In [ ]:
def sample_noise(G, noise_dim=10, n_samples=10000):
    X = np.random.uniform(0, 1, size=[n_samples, noise_dim])
    y = np.zeros((n_samples, 2))
    y[:, 1] = 1
    return X, y

In [ ]:
def train(GAN, G, D, epochs=500, n_samples=10000, noise_dim=10, batch_size=32, verbose=False, v_freq=50):
    d_loss = []
    g_loss = []
    e_range = range(epochs)
    if verbose:
        e_range = tqdm(e_range)
    for epoch in e_range:
        X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
        set_trainability(D, True)
        d_loss.append(D.train_on_batch(X, y)) #TREINAMENTO ***************************************************
        
        X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
        set_trainability(D, False)
        g_loss.append(GAN.train_on_batch(X, y))
        if verbose and (epoch + 1) % v_freq == 0:
            print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(epoch + 1, g_loss[-1], d_loss[-1]))
    return d_loss, g_loss

In [ ]:
d_loss, g_loss = train(GAN, G, D, verbose=True)

In [ ]:
ax = pd.DataFrame(
    {
        'Generative Loss': g_loss,
        'Discriminative Loss': d_loss,
    }
).plot(title='Training loss', logy=True)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")

In [ ]:
N_VIEWED_SAMPLES = 2
data_and_gen, _ = sample_data_and_gen(G, n_samples=N_VIEWED_SAMPLES)
pd.DataFrame(np.transpose(data_and_gen[N_VIEWED_SAMPLES:])).plot()